In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings

# Suppress specific FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils.validation")

pd.set_option('display.width', 1000)  # Change 1000 to the desired width
pd.set_option('display.max_columns', None)  # Show all columns

# Load the dataset
file_path = "slot_game_data.csv"
df = pd.read_csv(file_path)

# Normalize the features
scaler = MinMaxScaler()
df[['session_duration_hours', 'money_spent', 'avg_bet_amount']] = scaler.fit_transform(df[['session_duration_hours', 'money_spent', 'avg_bet_amount']])

# Define weights (you can adjust these based on domain knowledge)
weights = {
    'session_duration_hours': 0.4,
    'money_spent': 0.4,
    'avg_bet_amount': 0.2
}

# Calculate the score
df['score'] = (df['session_duration_hours'] * weights['session_duration_hours'] +
               df['money_spent'] * weights['money_spent'] +
               df['avg_bet_amount'] * weights['avg_bet_amount'])

# Aggregate scores by player_id and game_id (e.g., using mean)
df = df.groupby(['player_id', 'game_id'])['score'].mean().reset_index()

# Create a mapping of unique player and game IDs to indices
player_mapping = {player: idx for idx, player in enumerate(df['player_id'].unique())}
game_mapping = {game: idx for idx, game in enumerate(df['game_id'].unique())}

df['player_id'] = df['player_id'].map(player_mapping)
df['game_id'] = df['game_id'].map(game_mapping)

# Normalize the scores for each player so that the sum of their scores across different games adds up to 1
df['total_score'] = df.groupby('player_id')['score'].transform('sum')
df['normalized_score'] = df['score'] / df['total_score']

# Drop the total_score column as it's no longer needed
df = df.drop(columns=['total_score'])

# Check if the normalization was successful
print(df.head(20))

# Save dataframe to a new CSV file
df.to_csv('data.csv', index=False)


    player_id  game_id     score  normalized_score
0           0        0  0.298322          0.050966
1           0        1  0.465837          0.079584
2           0        2  0.506277          0.086493
3           0        3  0.341687          0.058375
4           0        4  0.667351          0.114012
5           0        5  0.280185          0.047867
6           0        6  0.306587          0.052378
7           0        7  0.242690          0.041462
8           0        8  0.482153          0.082372
9           0        9  0.361094          0.061690
10          0       10  0.237090          0.040505
11          0       11  0.387095          0.066132
12          0       12  0.438304          0.074881
13          0       13  0.389890          0.066610
14          0       14  0.448799          0.076674
15          1        1  0.197563          0.040415
16          1       15  0.252342          0.051620
17          1        2  0.323470          0.066171
18          1       16  0.14816